In [1]:
%%capture
!pip install unsloth "xformers==0.0.28.post2"
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"


In [2]:
%%capture
!pip install datasets
!pip install trl
!pip install transformers

In [1]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments


max_seq_length = 2048
dtype = None
load_in_4bit = True

fourbit_models = [
     "unsloth/llama-3-8b-bnb-4bit",
]




🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.12.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
INPUT_PATH_DATASET = "/content/drive/MyDrive/Colab Notebooks/dataset/transformed_trn.json"

In [6]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = load_dataset("json", data_files=INPUT_PATH_DATASET, split = "train")

train_dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/333776 [00:00<?, ? examples/s]

Map:   0%|          | 0/83445 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/333776 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/83445 [00:00<?, ? examples/s]

In [8]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 1000,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/333776 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [9]:
torch.cuda.empty_cache()
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 333,776 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 1,000
 "-____-"     Number of trainable parameters = 83,886,080
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ia-joaogabriel2024 (ia-joaogabriel2024-fiap). Use `wandb login --relogin` to force relogin


Step,Training Loss
1,2.605200
2,2.933000
3,2.626700
4,2.507100
5,1.814600
6,2.500000
7,2.237800
8,2.412600
9,2.195200
10,2.198700


Step,Training Loss
1,2.605200
2,2.933000
3,2.626700
4,2.507100
5,1.814600
6,2.500000
7,2.237800
8,2.412600
9,2.195200
10,2.198700


In [10]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "SUMMARIZE THIS PRODUCTS",
        "New York ABCs", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
SUMMARIZE THIS PRODUCTS

### Input:
New York ABCs

### Response:
Gr 3&ndash;5&ndash;In this clever alphabet book, readers will discover the wonders of New York City, from A to Z. Each letter has a corresponding picture that illustrates a different aspect of the city, such as the A for "architecture" that highlights the Empire State Building and the Z for "zoos" that features the Bronx Zoo. The text is simple and straightforward, and the illustrations are bold and colorful. The book is a great addition to a unit on New York City or any other city unit.&ndash;Elizabeth Doughty, Bloomfield High School, NJ&copy; Copyright


In [11]:
model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/unsloth_model") # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/unsloth_model")

('/content/drive/MyDrive/Colab Notebooks/unsloth_model/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/unsloth_model/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/unsloth_model/tokenizer.json')